In [165]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def g(data):
    return pd.read_csv(StringIO(data))

In [185]:
d="""movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy
6,Dummy 3 Title (1995),Comedy"""
df=gdf(d)
df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Dummy 3 Title (1995),Comedy


In [188]:
df.title.str.extract(r"\((\d+)\)")


,0
0,1995
1,1995
2,1995
3,1995
4,1995
5,1995


In [254]:
d="""date,time,value
NaN,1103,5  
NaN,0010,10
NaN,0100,6
NaN,0201,8
NaN,0015,7
12.03.2020,0400,4
NaN,0500,6
NaN,0800,3
NaN,1000,4
NaN,1140,5
12.03.2020,1200,2
NaN,0030,1
NaN,0112,0"""
df=pd.read_csv(StringIO(d), dtype='str')
df.head(8)

,date,time,value
0,NaN,1103,5
1,NaN,0010,10
2,NaN,0100,6
3,NaN,0201,8
4,NaN,0015,7
5,12.03.2020,0400,4
6,NaN,0500,6
7,NaN,0800,3


In [258]:
df['date']=pd.to_datetime(df.date, format='%d.%m.%Y')
df['newtime']=pd.to_datetime(df.time, format="%H%M")

In [263]:
#pd.to_timedelta(df.newtime.dt.time)
np.where(df.date.ne(pd.NaT),(df.date-df.newtime)/np.timedelta64(1, 'D'),pd.NaT)


array([nan, nan, nan, nan, nan, 43899.833333333336, nan, nan, nan, nan,
       43899.5, nan, nan], dtype=object)

In [244]:
pd.Timestamp('2020-03-12') - pd.Timestamp('1900-01-01 04:00:00')

Timedelta('43899 days 20:00:00')

In [252]:
pd.to_datetime([3792945600000000000], unit='ns')

DatetimeIndex(['2090-03-11 20:00:00'], dtype='datetime64[ns]', freq=None)

In [269]:
score = pd.DataFrame({'date':['2021-05-04','2021-10-05','2021-05-11']})
    
d1 = pd.to_datetime(score.date, format="%Y-%m-%d", errors='coerce')
d2 = pd.to_datetime(score.date, format="%Y-%d-%m", errors='coerce')

#score["date"] = np.where(d1.dt.weekday == 1, d1, d2)
d1.dt.weekday

0    1
1    1
2    1
Name: date, dtype: int64

In [271]:
mask = pd.to_datetime(score.date, errors='coerce', format = '%Y-%m-%d').isna()
score[mask] = pd.to_datetime(score[mask].date, format = '%Y-%d-%m').dt.date
score

,date
0,2021-05-04
1,2021-10-05
2,2021-05-11


In [281]:
score = pd.DataFrame({'date':['2021-05-04','2021-10-05','2021-05-11']})
d1 = pd.to_datetime(score.date, format="%Y-%m-%d", errors='coerce')
d2 = pd.to_datetime(score.date, format="%Y-%d-%m", errors='coerce')
d1.dt.day_name()
d2.dt.day_name()
#d2.dt.isocalendar().day

0    Monday
1    Monday
2    Friday
Name: date, dtype: object

In [284]:
d="""idx,start,end,user1,user2 
0,1,3,pass,pass
1,4,5,burger,burger 
2,6,7,nuggets,cola 
3,10,15,pass,burger"""
df=g(d)
df

,idx,start,end,user1,user2
0,0,1,3,pass,pass
1,1,4,5,burger,burger
2,2,6,7,nuggets,cola
3,3,10,15,pass,burger


In [285]:
df.columns

Index(['idx', 'start', 'end', 'user1', 'user2 '], dtype='object')

In [294]:
df[~df.filter(like='user').apply(set, axis=1).eq({'pass'})]

,idx,start,end,user1,user2
1,1,4,5,burger,burger
2,2,6,7,nuggets,cola
3,3,10,15,pass,burger


In [309]:
d="""datetime,column1,seasons
2021-01-01 01:00,11,Heating season
2021-05-01 02:00,10,Cooling season
2021-12-01 03:00,1,Heating season
2021-11-01 04:00,5,Heating season
2021-07-01 05:00,20,Cooling season"""
df=g(d)
df

df['datetime'] = pd.to_datetime(df.datetime)
df

,datetime,column1,seasons
0,2021-01-01 01:00:00,11,Heating season
1,2021-05-01 02:00:00,10,Cooling season
2,2021-12-01 03:00:00,1,Heating season
3,2021-11-01 04:00:00,5,Heating season
4,2021-07-01 05:00:00,20,Cooling season


In [304]:
df.datetime.dt.month
df.datetime.dt.month.isin(np.arange(5,10))
(df.datetime.dt.month.isin(np.arange(1,5))) | (df.datetime.dt.month.isin(np.arange(10,13)))

0     True
1    False
2     True
3    False
4    False
Name: datetime, dtype: bool

In [306]:
conditions = [
    df.datetime.dt.month.isin(np.arange(5,10)),
    (df.datetime.dt.month.isin(np.arange(1,5))) | (df.datetime.dt.month.isin(np.arange(10,13))),
]
choices = ['Cooling season','Heating season']
df['seasons'] = np.select(conditions, choices)
df

,datetime,column1,seasons,season
0,2021-01-01 01:00:00,11,Heating season,Heating season
1,2021-05-01 02:00:00,10,Cooling season,Cooling season
2,2021-12-01 03:00:00,1,Heating season,Heating season
3,2021-11-01 04:00:00,5,Heating season,Heating season
4,2021-07-01 05:00:00,20,Cooling season,Cooling season


In [307]:
conditions = [
    df.datetime.dt.month.isin(np.arange(5,10)),
]
choices = ['Cooling season']
df['seasons'] np.select(conditions, choices, default='Heating season')
df

,datetime,column1,seasons,season,seaso
0,2021-01-01 01:00:00,11,Heating season,Heating season,Heating season
1,2021-05-01 02:00:00,10,Cooling season,Cooling season,Cooling season
2,2021-12-01 03:00:00,1,Heating season,Heating season,Heating season
3,2021-11-01 04:00:00,5,Heating season,Heating season,Heating season
4,2021-07-01 05:00:00,20,Cooling season,Cooling season,Cooling season


In [308]:
df['seasons'] = np.where(df.datetime.dt.month.isin(np.arange(5,10)), 'Cooling season','Heating season')
df

,datetime,column1,seasons,season,seaso,s
0,2021-01-01 01:00:00,11,Heating season,Heating season,Heating season,Heating season
1,2021-05-01 02:00:00,10,Cooling season,Cooling season,Cooling season,Cooling season
2,2021-12-01 03:00:00,1,Heating season,Heating season,Heating season,Heating season
3,2021-11-01 04:00:00,5,Heating season,Heating season,Heating season,Heating season
4,2021-07-01 05:00:00,20,Cooling season,Cooling season,Cooling season,Cooling season


In [310]:
%timeit np.where((df["datetime"].dt.month >= 5) & (df["datetime"].dt.month <= 9),"Cooling season", "Heating season")

990 µs ± 68.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [311]:
%timeit np.where(df.datetime.dt.month.isin(np.arange(5,10)), 'Cooling season','Heating season')

400 µs ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [316]:
d="""postalcode,shipping_method,completed_orders
12345,post1,1
12345,post2,3
12345,post3,2
11123,post1,1
11123,post2,2"""
df=g(d)
df

,postalcode,shipping_method,completed_orders
0,12345,post1,1
1,12345,post2,3
2,12345,post3,2
3,11123,post1,1
4,11123,post2,2


In [323]:
df['amount_users_completed'] = df['completed_orders'] / (df.groupby(['postalcode', 'shipping_method']).completed_orders.transform(sum))
df

completed_orders  amount_users_completed
postalcode shipping_method                                          
11123      post1                           1                     1.0
           post2                           2                     1.0
12345      post1                           1                     1.0
           post2                           3                     1.0
           post3                           2                     1.0

In [324]:
d="""idUser,idMovie,Rating
1,1,4.1
1,2,5.1
2,1,4.4
3,5,2.8"""
df=g(d)
df

,idUser,idMovie,Rating
0,1,1,4.1
1,1,2,5.1
2,2,1,4.4
3,3,5,2.8


In [338]:
pd.crosstab(df.Rating, df.idMovie).astype(bool)

idMovie,1,2,5
Rating,,,
2.8,False,False,True
4.1,True,False,False
4.4,True,False,False
5.1,False,True,False


In [335]:
df.groupby(['Rating','idMovie']).size()

Rating  idMovie
2.8     5          1
4.1     1          1
4.4     1          1
5.1     2          1
dtype: int64

In [381]:
d="""timestamp,v
1,5
2,6
6,7
8,8"""
df=g(d)
df

,timestamp,v
0,1,5
1,2,6
2,6,7
3,8,8


In [376]:
df = df.set_index('timestamp')
df

,v
timestamp,
1,5
2,6
6,7
8,8


In [372]:
def test():
    all_time = pd.DataFrame({'timestamp':np.arange(df.timestamp.iloc[0], 
                                               df.timestamp.iloc[-1]+1)
                        })
    pd.merge_asof(all_time, df, on='timestamp', direction='backward')
%timeit test()

2.33 ms ± 166 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [383]:
def test1():
    df.reindex(np.arange(df.index[0], 1+df.index[-1])).ffill().reset_index().astype(int)
%timeit test1()

1.06 ms ± 54.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [63]:
timeline = pd.date_range(start="2027-01-01",
              end="2061-01-01",
              freq="H")
timeline = timeline[:-1]

df1 = pd.DataFrame()

In [64]:
for i in range(0, 34):
    df2 = pd.DataFrame()
    df2['value'] = np.random.randint(1, 6, 8900)
    df2['year'] = 2027 + i
    df1 = pd.concat([df1, df2])
df1 = df1.reset_index(drop=True)


,value,year
0,5,2027
1,2,2027
2,3,2027
3,4,2027
4,1,2027
...,...,...
302595,4,2060
302596,5,2060
302597,4,2060
302598,1,2060


In [65]:
timeline = timeline.to_frame()
timeline = timeline.rename(columns={(0):'date'})
timeline['tlyear'] = timeline.date.dt.year
timeline = timeline.reset_index(drop=True)
timeline

,date,tlyear
0,2027-01-01 00:00:00,2027
1,2027-01-01 01:00:00,2027
2,2027-01-01 02:00:00,2027
3,2027-01-01 03:00:00,2027
4,2027-01-01 04:00:00,2027
...,...,...
298051,2060-12-31 19:00:00,2060
298052,2060-12-31 20:00:00,2060
298053,2060-12-31 21:00:00,2060
298054,2060-12-31 22:00:00,2060


In [67]:
pd.concat([timeline,df1], join='inner', axis=1).drop('tlyear',1)

,date,value,year
0,2027-01-01 00:00:00,5,2027
1,2027-01-01 01:00:00,2,2027
2,2027-01-01 02:00:00,3,2027
3,2027-01-01 03:00:00,4,2027
4,2027-01-01 04:00:00,1,2027
...,...,...,...
298051,2060-12-31 19:00:00,1,2060
298052,2060-12-31 20:00:00,3,2060
298053,2060-12-31 21:00:00,2,2060
298054,2060-12-31 22:00:00,1,2060


In [48]:
timeline = pd.date_range(start="2028-02-01",
              end="2028-02-29", freq='D')

Int64Index([2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028,
            2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028, 2028,
            2028, 2028, 2028, 2028, 2028, 2028, 2028],
           dtype='int64')

In [49]:
df1=pd.DataFrame({
    'value':np.random.randint(1,6,30),
    'year':2028
})
df1.head()

,value,year
0,3,2028
1,1,2028
2,2,2028
3,5,2028
4,1,2028


In [50]:
timeline= timeline.to_frame()
timeline = tl.rename(columns={(0):'date'})
tl['tlyear'] = tl.date.dt.year
tl = tl.reset_index(drop=True)
tl.head()

,date,tlyear
0,2028-02-01,2028
1,2028-02-02,2028
2,2028-02-03,2028
3,2028-02-04,2028
4,2028-02-05,2028


In [53]:
pd.concat([tl,df1], join='inner', axis=1).drop('tlyear',1)

,date,value,year
0,2028-02-01,3,2028
1,2028-02-02,1,2028
2,2028-02-03,2,2028
3,2028-02-04,5,2028
4,2028-02-05,1,2028
5,2028-02-06,2,2028
6,2028-02-07,5,2028
7,2028-02-08,4,2028
8,2028-02-09,4,2028
9,2028-02-10,2,2028


In [108]:
d="""date,aa,bb,cc
2014-08-19 00:00,781,9.798,9.289
2014-08-19 00:10,782,10.004,9.382
2014-08-19 00:20,783,9.832,9.434
2014-08-19 00:30,784,10.019,9.593
2014-08-19 00:40,785,10.087,10.028"""
df=g(d)
df

,date,aa,bb,cc
0,2014-08-19 00:00,781,9.798,9.289
1,2014-08-19 00:10,782,10.004,9.382
2,2014-08-19 00:20,783,9.832,9.434
3,2014-08-19 00:30,784,10.019,9.593
4,2014-08-19 00:40,785,10.087,10.028


In [109]:
idx = df.groupby(pd.DatetimeIndex(data=df.date, freq='10min'))
df = idx.sum()
df

,aa,bb,cc
date,,,
2014-08-19 00:00:00,781,9.798,9.289
2014-08-19 00:10:00,782,10.004,9.382
2014-08-19 00:20:00,783,9.832,9.434
2014-08-19 00:30:00,784,10.019,9.593
2014-08-19 00:40:00,785,10.087,10.028


In [110]:
ts=['2014-08-19T00:00:00.000000000', '2014-08-19T00:10:00.000000000', '2014-08-19T00:20:00.000000000', '2015-08-16T23:40:00.000000000', '2015-08-16T23:50:00.000000000', '2015-08-17T00:00:00.000000000']

In [111]:
ts=pd.DatetimeIndex(ts)
ts

DatetimeIndex(['2014-08-19 00:00:00', '2014-08-19 00:10:00',
               '2014-08-19 00:20:00', '2015-08-16 23:40:00',
               '2015-08-16 23:50:00', '2015-08-17 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [112]:
df = df.reindex(ts)

In [113]:
df

,aa,bb,cc
2014-08-19 00:00:00,781.0,9.798,9.289
2014-08-19 00:10:00,782.0,10.004,9.382
2014-08-19 00:20:00,783.0,9.832,9.434
2015-08-16 23:40:00,NaN,NaN,NaN
2015-08-16 23:50:00,NaN,NaN,NaN
2015-08-17 00:00:00,NaN,NaN,NaN


In [117]:
ts = ['2014-08-19 00:00:00', '2014-08-19 00:10:00', '2014-08-19 00:20:00', '2014-08-19 00:30:00', '2014-08-19 00:40:00', '2014-08-19 00:50:00', '2015-08-16 22:30:00', '2015-08-16 22:40:00', '2015-08-16 22:50:00', '2015-08-16 23:00:00', '2015-08-16 23:10:00', '2015-08-16 23:20:00', '2015-08-16 23:30:00', '2015-08-16 23:40:00', '2015-08-16 23:50:00', '2015-08-17 00:00:00']
ts=pd.DatetimeIndex(ts)
ts

DatetimeIndex(['2014-08-19 00:00:00', '2014-08-19 00:10:00',
               '2014-08-19 00:20:00', '2014-08-19 00:30:00',
               '2014-08-19 00:40:00', '2014-08-19 00:50:00',
               '2015-08-16 22:30:00', '2015-08-16 22:40:00',
               '2015-08-16 22:50:00', '2015-08-16 23:00:00',
               '2015-08-16 23:10:00', '2015-08-16 23:20:00',
               '2015-08-16 23:30:00', '2015-08-16 23:40:00',
               '2015-08-16 23:50:00', '2015-08-17 00:00:00'],
              dtype='datetime64[ns]', freq=None)

In [116]:
df = df.reindex(ts)
df

,aa,bb,cc
2014-08-19 00:00:00,781.0,9.798,9.289
2014-08-19 00:10:00,782.0,10.004,9.382
2014-08-19 00:20:00,783.0,9.832,9.434
2014-08-19 00:30:00,NaN,NaN,NaN
2014-08-19 00:40:00,NaN,NaN,NaN
2014-08-19 00:50:00,NaN,NaN,NaN
2015-08-16 22:30:00,NaN,NaN,NaN
2015-08-16 22:40:00,NaN,NaN,NaN
2015-08-16 22:50:00,NaN,NaN,NaN
2015-08-16 23:00:00,NaN,NaN,NaN


In [166]:
df = pd.DataFrame({
    'id':[11, 12],
    'age_in_day':[22643, 10262]
})
#age_Series = (df['age_in_day'] / 365)
##age_Series
#df['age_in_years'] = age_Series
df

,id,age_in_day
0,11,22643
1,12,10262


In [170]:
#df['age_in_years'] = pd.to_timedelta(df.age_in_day, unit='D') / np.timedelta64(1,'Y')